In [ ]:
import polars as pl

# Load up metadata from "DNA from hoiho genomesv2.csv"
metadata = pl.read_csv(
    #"Hoiho_Genomes_Cleaned_Nov4_24_24.csv", separator="\t"
    "Hoiho_Genomes_24Feb2024_JGG_3Pops.csv", separator="\t"
)

metadata = metadata.with_columns(
    pl.col("ID").replace("P29 ", "P29").alias("ID")
)

# Next is C101/CE9
metadata = metadata.with_columns(
    pl.col("ID").replace("C101/CE9", "CE9").alias("ID")
)

In [ ]:
metadata.columns

In [ ]:
# Print out just ID and Population3 fields as space separated
df = metadata.select([pl.col("ID"), pl.col("Population3")])
df.write_csv("pop_assignments.txt", separator=" ")
df = metadata.select([pl.col("ID"), pl.col("Location")])
df.write_csv("pop_assignments_comprehensive.txt", separator=" ")
df


In [ ]:
# Read in fastStructure/plink2.fam file
fam = pl.read_csv("fastStructure/plink2.fam", separator="\t", has_header=False)
# Get column 2 only
samples = fam.get_column("column_2").to_list()

In [ ]:
# For each sample, get the population from metadata
populations = []
for sample in samples:
    population = metadata.filter(pl.col("ID") == sample).select("Location").get_column("Location").to_list()[0]
    populations.append(population)

In [ ]:
populations

In [ ]:
# Write out to a new file
with open("fastStructure/locations.txt", "w") as f:
    for population in populations:
        f.write(format(population) + "\n")